Прежде чем сдать эту работу, убедитесь, что всё выполняется так, как ожидается.
Сначала перезапустите ядро (в меню выберите Kernel → Restart), а затем запустите все ячейки (в меню выберите Cell → Run All).

Убедитесь, что вы заполнили все места, где указано YOUR CODE HERE или "YOUR ANSWER HERE", а также указали своё имя ниже.

In [ ]:
NAME = ""

Убедитесь, что используете версию Python > 3. Все тетрадки отрабатывают точно на версии 3.10.10.

In [ ]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

---

## Домашнее задание 2. Градиентный спуск своими руками

### О задании

В данном задании необходимо реализовать обучение линейной регрессии с помощью различных модификаций градиентного спуска. **В ячейках ниже в ноутбуке** вам нужно будет реализовать несколько классов для различных вариаций градиентного спуска, а именно:
* `VanillaGradientDescent`
* `StochasticGradientDescent`
* `MomentumDescent`
* `Adam`

Также вам необходимо будет реализовать класс `LinearRegression` для обучения линейной регрессии (и, разумеется, предсказания целевой переменной на основе обученной модели).

## Задание 1. Реализация градиентного спуска

В этом задании вам предстоит написать собственные реализации различных подходов к градиентному спуску с опорой на шаблоны в ячейках ниже.

**Все реализуемые методы должны быть векторизованы!**

### Напоминание про градиентный спуск

Основное свойство антиградиента &ndash; он указывает в сторону *наискорейшего* убывания функции в данной точке. Соответственно, будет логично стартовать из некоторой точки, сдвинуться в сторону антиградиента,
пересчитать антиградиент и снова сдвинуться в его сторону и т.д. Запишем это более формально.

Пусть $w_0$ &ndash; начальный набор параметров (например, нулевой или сгенерированный из некоторого
случайного распределения). Тогда ванильный градиентный спуск состоит в повторении следующих шагов до сходимости:

$$
    w_{k + 1} = w_{k} - \eta_{k} \nabla_{w} Q(w_{k}).
$$

Здесь $\eta_{k}$ обозначает длину шага на $k$-ой итерации (learning rate), а $Q(w)$ - функцию потерь (loss function).

### Задание 1.1. Learning Rate Schedules (0.06 балла)

**Learning rate** (скорость обучения) $\eta$ — это гиперпараметр, который определяет величину шага при обновлении весов модели в направлении антиградиента. Если learning rate слишком большой, градиентный спуск может «перепрыгивать» через минимум и расходиться; если слишком маленький — обучение будет идти крайне медленно. Поэтому часто используют **расписания** (schedules), которые адаптивно меняют $\eta$ в процессе обучения: например, начинают с большого шага для быстрого продвижения и постепенно уменьшают его для более точной сходимости.

С помощью  класса `LearningRateSchedule` мы на каждой итерации градиентного спуска будем получать соответствующий `learning_rate` $\eta_k$.

В ячейке ниже уже реализован класс `ConstantLR`, который на каждой итерации возвращает один и тот же заранее заданный шаг. Ваша задача в этом пункте – реализовать `TimeDecayLR`, который мы будем использовать для обучения линейной регрессии. Формула очередного шага должна выглядеть следующим образом:
$$
    \eta_{k} = \lambda \left(\dfrac{s_0}{s_0 + k}\right)^p
$$

На практике достаточно настроить параметр $\lambda$, а остальным выставить параметры по умолчанию: $s_0 = 1, \, p = 0.5.$

In [ ]:
import numpy as np
from abc import ABC, abstractmethod
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

class LearningRateSchedule(ABC):
    @abstractmethod
    def get_lr(self, iteration: int) -> float:
        pass


class ConstantLR(LearningRateSchedule):
    def __init__(self, lr: float):
        self.lr = lr

    def get_lr(self, iteration: int) -> float:
        return self.lr


class TimeDecayLR(LearningRateSchedule):
    def __init__(self, lambda_: float = 0.1):
        self.s0 = 1
        self.p = 0.5
        self.lambda_ = lambda_

    def get_lr(self, iteration: int) -> float:
        # YOUR CODE HERE
        raise NotImplementedError()

Выполните ячейку ниже для проверки. 

In [ ]:
lr = TimeDecayLR(lambda_=1.0)
assert abs(lr.get_lr(0) - 1.0) < 1e-9, "get_lr(0) должен быть 1.0"
assert lr.get_lr(1) < lr.get_lr(0), "Шаг должен убывать с итерацией"
assert abs(lr.get_lr(3) - (1.0 * (1/(1+3))**0.5)) < 1e-9

assert abs(lr.get_lr(99) - (1.0 * (1/100)**0.5)) < 1e-9
lr2 = TimeDecayLR(lambda_=0.5)
assert abs(lr2.get_lr(0) - 0.5) < 1e-9

### Задание-примечание 1.1. Родительский класс BaseDescent (0 баллов).

Ниже приведен шаблон класса `BaseDescent` – родительского класса для модификаций градиентного спуска, от которого будут наследоваться другие классы (`VanillaGradientDescent`, `StochasticGradientDescent`, `MomentumDescent` и `Adam`). Более подробно про наследование классов в Python можно прочитать
* Наследование: https://docs.python.org/3/tutorial/classes.html#inheritance
* Абстрактные классы: https://docs.python.org/3/library/abc.html

В классе `BaseDescent` **все методы уже реализованы**. Цель этого задания – внимательно ознакомиться с тем, как устроен этот класс.

Обратите внимание на атрибут `self.iteration`, отвечающий за номер итерации алгоритма спуска. Как раз с помощью него (и `self.lr_schedule`) мы и будем получать `learning_rate` на соответствующей итерации алгоритма. Функция `update_weights` должна обновлять веса модели `self.model.w`, а также возвращать величину обновления $w_{k + 1} - w_k$

**Обратите внимание**

Все реализуемые вами классы спуска в задании - это *универсальные* оптимизаторы. Они не должны считать градиенты конкретной функции потерь внутри себя.

Для вычисления градиента они всегда обращаются к модели, с которой работают:

```
gradient = self.model.compute_gradients(X_batch, y_batch)
```

Чтобы это работало, уже на данном этапе должны быть реализованы в классе `LinearRegression`:

* `compute_gradients(X, y)` для MSE (в дальнейшем, в Задании 7, сюда добавляется член L2-регуляризации),
* `compute_loss(X, y)` для MSE (аналогично с учётом L2 при необходимости).

Если идёте строго по порядку, реализуйте эти MSE-версии в начале Задания 2.1, а затем вернитесь к заданиям 1.2–1.6 — код оптимизаторов менять не придётся.

Такой подход позволяет свободно менять функцию потерь, не переписывая алгоритмы оптимизации.

In [ ]:
class BaseDescent(ABC):
    def __init__(self, lr_schedule: LearningRateSchedule = TimeDecayLR()):
        self.lr_schedule = lr_schedule
        self.iteration = 0
        self.model = None

    def set_model(self, model):
        self.model = model

    @abstractmethod
    def update_weights(self):
        pass

    def step(self):
        update = self.update_weights()
        self.iteration += 1
        return update

### Задание 1.2. Полный градиентный спуск VanillaGradientDescent (0.5 балла).

Реализуйте полный градиентный спуск, заполнив пропуски в классе `VanillaGradientDescent` в ячейке ниже. Напомним, что шаг классического градиентного спуска выглядит следующим образом:

$$
    w_{k + 1} = w_{k} - \eta_{k} \nabla_{w} Q(w_{k}).
$$

**Важно**: Здесь и далее функция `update_weights` должна возвращать разницу между 

$w_{k + 1}$ и $w_{k}$: $\quad w_{k + 1} - w_{k} = -\eta_{k} \nabla_{w} Q(w_{k})$

Кроме того, соответственно своему названию, она должна обновлять веса модели `model.w`.

In [ ]:
class VanillaGradientDescent(BaseDescent):
    def update_weights(self):
        X_train = self.model.X_train
        y_train = self.model.y_train
        gradient = self.model.compute_gradients(X_train, y_train)
        # YOUR CODE HERE
        raise NotImplementedError()

### Напоминание про SGD (стохастических градиентный спуск)

Как правило, в задачах машинного обучения функционал $Q(w)$ представим в виде суммы $\ell$ функций:

$$
    Q(w)
    =
    \frac{1}{\ell}
    \sum_{i = 1}^{\ell}
        q_i(w).
$$

В нашем домашнем задании отдельные функции $q_i(w)$ соответствуют ошибкам на отдельных объектах.

Проблема метода градиентного спуска состоит в том, что на каждом шаге необходимо вычислять градиент всей суммы (будем его называть полным градиентом):

$$
    \nabla_w Q(w)
    =
    \frac{1}{\ell}
    \sum_{i = 1}^{\ell}
        \nabla_w q_i(w).
$$

Это может быть очень трудоёмко при больших размерах выборки. В то же время точное вычисление градиента может быть не так уж необходимо &ndash; как правило, мы делаем не очень большие шаги в сторону антиградиента, и наличие в нём неточностей не должно сильно сказаться на общей траектории.

Оценить градиент суммы функций можно средним градиентов случайно взятого подмножества функций:

$$
    \nabla_{w} Q(w_{k}) \approx \dfrac{1}{|B|}\sum\limits_{i \in B}\nabla_{w} q_{i}(w_{k}),
$$
где $B$ - это случайно выбранное подмножество индексов, обычно называемое **батчом**.

Оценка $\frac{1}{|B|} \sum \limits_{i \in B} \nabla_w q_i(w_k)$ называется **стохастическим градиентом** функции потерь, а получившийся метод называют методом **стохастического градиентного спуска** или просто SGD.

### Задание 1.3. Стохастический градиентный спуск StochasticGradientDescent (0.7 баллов).

Реализуйте стохастический градиентный спуск, заполнив пропуски в классе `StochasticGradientDescent`. Шаг оптимизации:

$$
    w_{k + 1} = w_{k} - \eta_{k} \dfrac{1}{|B|}\sum\limits_{i \in B}\nabla_{w} q_{i}(w_{k}).
$$

Размер батча будет являться **гиперпараметром** метода и передаваться в конструктор класса `__init__(...)`. Семплировать индексы батча объектов $B$ можно с повторениями (через np.random.randint) - это допустимо и даёт несмещённую оценку градиента. По желанию можно без повторений (np.random.choice(..., replace=False) или через пермутацию по эпохам).

In [ ]:
np.random.seed(20)
class StochasticGradientDescent(BaseDescent):
    def __init__(self, lr_schedule: LearningRateSchedule = TimeDecayLR(), batch_size=1):
        super().__init__(lr_schedule)
        self.batch_size = batch_size

    def update_weights(self):
        X_train = self.model.X_train
        y_train = self.model.y_train
        # YOUR CODE HERE
        raise NotImplementedError()

### Напоминание про метод инерции (или метод моментов)

Может оказаться, что направление антиградиента сильно меняется от шага к шагу. Например, если линии уровня функционала сильно вытянуты, то из-за ортогональности градиента линиям уровня он будет менять направление на почти противоположное на каждом шаге. Такие осцилляции будут вносить сильный шум в движение, и процесс оптимизации займёт много итераций. Чтобы избежать этого, можно усреднять векторы антиградиента с нескольких предыдущих шагов &ndash; в этом случае шум уменьшится, и такой средний вектор будет указывать в сторону общего направления движения. Введём для этого вектор инерции:

\begin{align}
    &h_0 = 0, \\
    &h_{k + 1} = \alpha h_{k} + \eta_k \nabla_w Q(w_{k})
\end{align}

Здесь $\alpha$ &ndash; параметр метода, определяющей скорость затухания градиентов с предыдущих шагов. Разумеется, вместо вектора градиента может быть использована его аппроксимация (например, в случае **стохастического градиентного спуска**). Чтобы сделать шаг градиентного спуска, просто сдвинем предыдущую точку на вектор инерции:

$$
    w_{k + 1} = w_{k} - h_{k + 1}.
$$

Заметим, что если по какой-то координате градиент постоянно меняет знак, то в результате усреднения градиентов в векторе инерции эта координата окажется близкой к нулю. Если же по координате знак градиента всегда одинаковый, то величина соответствующей координаты в векторе инерции будет большой, и мы будем делать большие шаги в соответствующем направлении.

### Задание 1.5 Метод Momentum - MomentumDescent (1 балл).

Реализуйте градиентный спуск с методом инерции заполнив пропуски в классе `MomentumDescent`. Шаг оптимизации:

\begin{align}
    &h_0 = 0, \\
    &h_{k + 1} = \beta h_{k} + \eta_k \nabla_w Q(w_{k}) \\
    &w_{k + 1} = w_{k} - h_{k + 1}.
\end{align}

$\beta$ является гиперпараметром метода, однако в данном домашнем задании мы зафиксируем её за вас $\beta = 0.9$.

In [ ]:
class MomentumDescent(BaseDescent):
    def __init__(self, lr_schedule: LearningRateSchedule = TimeDecayLR(), beta=0.9):
        super().__init__(lr_schedule)
        self.beta = beta
        self.velocity = None

    def update_weights(self):
        X_train = self.model.X_train
        y_train = self.model.y_train
        # YOUR CODE HERE
        raise NotImplementedError()

### Напоминание про AdaGrad, RMSprop и Adam

Градиентный спуск очень чувствителен к выбору длины шага. Если шаг большой, то есть риск, что мы будем перескакивать через точку минимума; если же шаг маленький, то для нахождения минимума потребуется много итераций. При этом нет способов заранее определить правильный размер шага &ndash; к тому же, схемы с постепенным уменьшением шага по мере итераций могут тоже плохо работать.

В методе AdaGrad предлагается сделать свою длину шага для каждой компоненты вектора параметров. Идея проста: мы будем "копить" сумму квадратов градиентов и делить очередной градиент на корень из этой суммы. Таким образом, обновление весов с большими градиентами будет тормозиться, а с маленькими наоборот получать большие шаги. Формула обновлени будет выглядить так:

\begin{align}
    &G_{kj} = G_{k-1,j} + (\nabla_w Q(w_{k - 1}))_j^2; \\
    &w_{jk} = w_{j,k-1} - \frac{\eta_t}{\sqrt{G_{kj}} + \varepsilon} (\nabla_w Q(w_{k - 1}))_j.
\end{align}

Здесь $\varepsilon$ небольшая константа, которая предотвращает деление на ноль.

В данном методе можно зафиксировать длину шага (например, $\eta_k = 0.01$) и не подбирать её в процессе обучения **(обратите внимание, что в данном домашнем задании длина шага не фиксируется)**. Отметим, что данный метод подходит для разреженных задач, в которых у каждого объекта большинство признаков равны нулю. Для признаков, у которых ненулевые значения встречаются редко, будут делаться большие шаги; если же какой-то признак часто является ненулевым, то шаги по нему будут небольшими.

У метода AdaGrad есть большой недостаток: переменная $G_{kj}$ монотонно растёт, из-за чего шаги становятся всё медленнее и могут остановиться ещё до того, как достигнут минимум функционала. Проблема решается в методе RMSprop, где используется экспоненциальное затухание градиентов:

$$
    G_{kj} = \alpha G_{k-1,j} + (1 - \alpha) (\nabla_w Q(w^{(k-1)}))_j^2.
$$

В этом случае размер шага по координате зависит в основном от того, насколько
быстро мы двигались по ней на последних итерациях.

Можно объединить идеи описанных выше методов: накапливать градиенты со всех прошлых шагов для
избежания осцилляций (метод инерции), а также делать адаптивную длину шага по каждому параметру (`RMSProp`). Таким образом, мы получим метод `Adam` с той лишь разницей, что в методе `Adam` дополнительно делается нормировка накопленных градиентов и квадратов градиентов для устранения смещения.

### Задание 1.6. Метод Adam (Adaptive Moment Estimation) (1.2 балла).

Реализуйте градиентный спуск с методом Adam, заполнив пропуски в классе `Adam`. Шаг оптимизации:

\begin{align}
    &m_0 = 0, \quad v_0 = 0; \\ \\
    &m_{k + 1} = \beta_1 m_k + (1 - \beta_1) \nabla_w Q(w_{k}); \\ \\
    &v_{k + 1} = \beta_2 v_k + (1 - \beta_2) \left(\nabla_w Q(w_{k})\right)^2; \\ \\
    &\widehat{m}_{k} = \dfrac{m_k}{1 - \beta_1^{k}}, \quad \widehat{v}_{k} = \dfrac{v_k}{1 - \beta_2^{k}}; \\ \\
    &w_{k + 1} = w_{k} - \dfrac{\eta_k}{\sqrt{\widehat{v}_{k + 1}} + \varepsilon} \widehat{m}_{k + 1}.
\end{align}

$\beta_1 = 0.9, \beta_2 = 0.999$ и $\varepsilon = 10^{-8}$ будут зафиксированы за вас.

In [ ]:
class Adam(BaseDescent):
    def __init__(self, lr_schedule: LearningRateSchedule = TimeDecayLR(), beta1=0.9, beta2=0.999, eps=1e-8):
        super().__init__(lr_schedule)
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.m = None
        self.v = None

    def update_weights(self):
        X_train = self.model.X_train
        y_train = self.model.y_train
        # YOUR CODE HERE
        raise NotImplementedError()

## Задание 2. Линейная регресия

Напомним, что функция потерь MSE записывается как:

$$
    Q(w) = \frac{1}{n} \sum \limits_{i = 1}^n (y_i - \langle x_i, w \rangle)^2 = \frac{1}{n} \| X w - y \|^2
$$

где $n$ – количество объектов в выборке, $X \in \mathbb{R}^{n \times d}$ – матрица "объект-признак", а $y \in \mathbb{R}^n$ – целевая переменная. Через $x_i$ обозначается $i$-ая строчка матрицы $X$, отвечающая за $i$-й объект выборки.

Выпишите ниже (подсмотрев в семинар или решив самостоятельно) градиент для функции потерь MSE в матричном виде.

### Задание 2.1. Градиент MSE в матричном виде (0.04 балла).
Напомним, что функция потерь MSE записывается как:

$$
    Q(w) = \frac{1}{n} \sum \limits_{i = 1}^n (y_i - \langle x_i, w \rangle)^2 = \frac{1}{n} \| X w - y \|^2
$$

где $n$ – количество объектов в выборке, $X \in \mathbb{R}^{n \times d}$ – матрица "объект-признак", а $y \in \mathbb{R}^n$ – целевая переменная. Через $x_i$ обозначается $i$-ая строчка матрицы $X$, отвечающая за $i$-й объект выборки.

Выпишите ниже (подсмотрев в семинар или решив самостоятельно) градиент для функции потерь MSE в матричном виде.

YOUR ANSWER HERE

### Задание 2.2. Решение методом градиентного спуска (1 балл)

В этом задании вам предстоит написать свою реализацию линейной регрессии, обучаемой с использованием градиентного спуска, с опорой на шаблон в ячейках выше — класс **LinearRegression**. По сути линейная регрессия будет оберткой, которая запускает обучение

Необходимо соблюдать следующие условия:

* Все вычисления должны быть векторизованы;
* Циклы средствами python допускаются только для итераций градиентного спуска;
* В качестве критерия останова необходимо использовать (одновременно):
    * Квадрат евклидовой нормы разности весов на двух соседних итерациях меньше `tolerance`;
    * Разность весов содержит наны;
    * Достижение максимального числа итераций `max_iter`.
* Будем считать, что все данные, которые поступают на вход имеют столбец единичек последним столбцом;
* Используйте `optimizer.step()` для обновления весов
* Чтобы проследить за сходимостью оптимизационного процесса будем использовать `loss_history`, в нём будем хранить значения функции потерь до каждого шага, начиная с нулевого (до первого шага по антиградиенту) и значение функции потерь после оптимизации.

In [ ]:
from dataclasses import dataclass
from enum import auto, Enum
from typing import Optional

class LossFunction(Enum):
    MSE = auto()

In [ ]:
np.random.seed(42)

class LinearRegression:
    def __init__(
        self,
        optimizer: Optional[BaseDescent | str] = None,
        l2_coef: float = 0.0,
        tolerance: float = 1e-6,
        max_iter: int = 1000,
        loss_function: LossFunction = LossFunction.MSE
    ):
        self.optimizer = optimizer
        if isinstance(optimizer, BaseDescent):
            self.optimizer.set_model(self)
        self.l2_coef = l2_coef
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.loss_function = loss_function
        self.w = None
        self.X_train = None
        self.y_train = None
        self.loss_history = []

    def predict(self, X: np.ndarray) -> np.ndarray:
        # Подсказка: предсказание линейной модели — это матричное умножение датасета на веса
        # YOUR CODE HERE
        raise NotImplementedError()

    def compute_gradients(self, X: np.ndarray, y: np.ndarray) -> np.ndarray:
        # Выше вы выводили формулу градиента для MSE
        # Не забудьте про слагаемое регуляризации: + l2_coef * w (для 7 задания)
        if self.loss_function is LossFunction.MSE:
            # YOUR CODE HERE
            raise NotImplementedError()
        else:
            raise ValueError("Unsupported loss function")

    def compute_loss(self, X: np.ndarray, y: np.ndarray) -> float:
        if self.loss_function is LossFunction.MSE:
            # YOUR CODE HERE
            raise NotImplementedError()
        else:
            raise ValueError("Unsupported loss function")

    def fit(self, X: np.ndarray, y: np.ndarray):
        self.X_train, self.y_train = X, y
        n, d = X.shape
        self.loss_history = []

        # Аналитическое решение (optimizer is None)
        # можете пока игнорировать, это следующее задание
        if self.optimizer is None:
            # YOUR CODE HERE
            raise NotImplementedError()
            self.loss_history.append(self.compute_loss(X, y))
            return

        # Градиентный спуск
        #   1. Инициализируйте веса нулями
        #   2. Запишите начальный loss в loss_history
        #   3. В цикле до max_iter:
        #      - обновите веса с помощью функции `optimizer.step()`
        #      - проверьте критерии останова
        #      - если не остановились — запишите текущий loss в loss_history
        if isinstance(self.optimizer, BaseDescent):
            # YOUR CODE HERE
            raise NotImplementedError()

### Задание 2.3. Аналитическое решение (0.5 балла)

Но, как мы помним из лекции, помимо решения при помощи градиентного спуска, для ряда функций потерь можно выписать в том числе аналитическое решение. Давайте сперва вспомним, как оно выглядит для MSE. Выведите оптимальную формулу для $w$, держа в памяти формулу MSE, и дополните класс `LinearRegression`

$$\text{MSE} = \| X w - y \|^2$$
$$ w = $$

**Вопрос**: Как мы помним, у аналитического решения есть минусы, какие кстати?

**Ответ**: ответ не оценивается. Вспомните, что мы обсуждали на лекции


Теперь добавьте это решение в блок `LinearRegression` для случая, когда `optimizedr=None`

In [ ]:
import numpy as np

np.random.seed(42)

num_objects = 100
dimension = 5

x = np.random.rand(num_objects, dimension)
y = np.random.rand(num_objects)

In [ ]:
from sklearn.metrics import mean_squared_error as mse
import sklearn

np.random.seed(42)

sklearn_linreg = sklearn.linear_model.LinearRegression(fit_intercept=False)
sklearn_linreg.fit(x, y)

your_linreg = LinearRegression(optimizer=None)
your_linreg.fit(x, y)

assert abs(mse(your_linreg.predict(x), y) - mse(sklearn_linreg.predict(x), y)) < 1e-12, "Не повезло, попробуйте еще раз"

x2 = np.random.rand(50, 4); y2 = np.random.rand(50)
lr2 = LinearRegression(optimizer=None); lr2.fit(x2, y2)
assert lr2.w.shape == (4,)
sk2 = sklearn.linear_model.LinearRegression(fit_intercept=False).fit(x2, y2)
assert abs(mse(lr2.predict(x2), y2) - mse(sk2.predict(x2), y2)) < 1e-10

## Задание 3. Проверка кода (0 баллов)

Данная секция нужна для того, чтобы убедиться в правильности реализации методов спуска и класса `LinearRegression`.

In [ ]:
np.random.seed(20)

max_iter = 10
tolerance = 0
num_objects = 100
dimension = 10
x = np.random.rand(num_objects, dimension)
y = x @ np.random.rand(dimension) + 2 + np.random.random(num_objects)

In [ ]:
optimizer = VanillaGradientDescent()
model = LinearRegression(optimizer=optimizer, max_iter=max_iter, tolerance=tolerance)
model.fit(x, y)
y_pred = model.predict(x)
assert y_pred.shape == y.shape, "VanillaGradientDescent: Prediction shape does not match"
assert len(model.loss_history) == max_iter + 1, "Loss history failed"
assert model.loss_history[-1] < model.loss_history[0], "Loss history failed"

In [ ]:
optimizer = StochasticGradientDescent(batch_size=5)
model = LinearRegression(optimizer=optimizer, max_iter=max_iter, tolerance=tolerance)
model.fit(x, y)
y_pred = model.predict(x)
assert y_pred.shape == y.shape, "StochasticGradientDescent: Prediction shape does not match"
assert len(model.loss_history) == max_iter + 1, "Loss history failed"
assert model.loss_history[-1] < model.loss_history[0], "Loss history failed"

In [ ]:
optimizer = MomentumDescent()
model = LinearRegression(optimizer=optimizer, max_iter=max_iter, tolerance=tolerance)
model.fit(x, y)
y_pred = model.predict(x)
assert y_pred.shape == y.shape, "MomentumDescent: Prediction shape does not match"
assert len(model.loss_history) == max_iter + 1, "Loss history failed"
assert model.loss_history[-1] < model.loss_history[0], "Loss history failed"

In [ ]:
optimizer = Adam()
model = LinearRegression(optimizer=optimizer, max_iter=max_iter, tolerance=tolerance)
model.fit(x, y)
y_pred = model.predict(x)
assert y_pred.shape == y.shape, "Adam: Prediction shape does not match"
assert len(model.loss_history) == max_iter + 1, "Loss history failed"
assert model.loss_history[-1] < model.loss_history[0], "Loss history failed"

In [ ]:
from sklearn.linear_model import LinearRegression as SkLinearRegression
from sklearn.metrics import mean_squared_error

sk_model = SkLinearRegression(fit_intercept=False)
sk_model.fit(x, y)
y_sk = sk_model.predict(x)

my_model = LinearRegression(optimizer=None)
my_model.fit(x, y)
y_my = my_model.predict(x)

assert y_sk.shape == y_my.shape == y.shape, "LinearRegression: Prediction shape does not match target"
assert abs(mean_squared_error(y_sk, y) - mean_squared_error(y_my, y)) < 1e-10, "LinearRegression: MSEs differ"

## Задание 4. Работа с данными

Представьте, что вы — инженер-эколог в крупной аналитической компании, которая помогает автопроизводителям и регуляторам понимать реальные выбросы углекислого газа.

Вам передали большой набор данных об автомобилях разных лет, классов, типов топлива и двигателей. Ваша задача — построить модель, которая по техническим характеристикам машины с максимально возможной точностью предскажет, сколько граммов CO₂ будет выбрасывать этот автомобиль на каждую милю пути.

В этом задании вам необходимо проанализировать данные для будущего обучения. Напрямую за EDA оценка не ставится, но от него зависит будущая метрика. За метрику оценка ставится.

Целевой признак (то, что предсказываем):

- **co2TailpipeGpm** — выбросы CO₂ из выхлопной трубы, г/милю

Доступные признаки:

- displ — рабочий объём двигателя, литры
- cylinders — количество цилиндров
- year — год выпуска модели
- comb — комбинированный расход топлива (город + трасса), миль на галлон
- highway — расход на трассе, MPG
- city — расход в городе, MPG
- fuelCost — расчётная годовая стоимость топлива, $
- fuelType1 — тип основного топлива 
- drive — тип привода
- trany — тип трансмиссии 
- VClass — класс автомобиля по классификации EPA 
- tCharger — наличие турбонаддува (T = есть)
- sCharger — наличие механического нагнетателя (S = есть)
- guzzler — автомобиль попадает под «gas guzzler tax»
- startStop — наличие системы start-stop (Y = есть)

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Классы ConstantLR, TimeDecayLR, VanillaGradientDescent, StochasticGradientDescent,
# MomentumDescent, Adam, SAGDescent и LinearRegression определены выше в ноутбуке

sns.set(style='darkgrid')

У вас уже есть разделенные выборки на train и test:

In [ ]:
x_train = pd.read_csv('train.csv')
x_test = pd.read_csv('test_with_target.csv')

In [ ]:
x_train.head()

Добавляем в данные единичную колонку `bias`, чтобы не делать отдельные параметр $b$ для свободного члена модели.

In [ ]:
x_train['bias'] = 1
x_test['bias'] = 1

y_train = x_train['co2TailpipeGpm']
x_train = x_train.drop(columns=['co2TailpipeGpm'])

y_test = x_test['co2TailpipeGpm']
x_test = x_test.drop(columns=['co2TailpipeGpm'])

Теперь вам необходимо сделать EDA для последующего обучения:

* Постройте распределение таргетовой метрики. Посмотрите, необходимо ли его логарфмировать? 
* Постройте графики между фичами и таргетовой метрикой. Здесь вам очень поможет `sns.pairplot`. Есть ли какая-то очевидная зависимость между таргетом и какой-то из фич? Если да, эта зависимость линейная? Есть ли линейно зависимые признаки?
* Постройте распределение категориальных фич. Можно ли какие-то из них упростить? Содержательны ли они? Можно ли из них создать новые фичи? 
* Посмотрите, есть ли пропуски в данных. Подумайте, что с ними делать как для численных признаков, так и для категориальных

* Если вы добавляете\удаляете\меняете фичу на x_train, не забывайте то же самое делать и для x_test.

* В следующем задании вам необходимо будет подобрать гиперпараметры, поэтому разделите выборку x_train на x_train и x_valid c test_size=0.2. На x_valid вы будете подбирать гиперпараметры. Не забудьте по честное деление без даталиков.

In [ ]:
# используйте названия для датасета x_train, x_val, y_train, y_val
# YOUR CODE HERE
raise NotImplementedError()


После обработки фич разделите их на категориальные, числовые и другие (например, bias):

In [ ]:
x_train.columns

In [ ]:
# categorical = []
# numeric = []
# other = ["bias"]

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
x_train = x_train[categorical + numeric + other]
x_val = x_val[categorical + numeric + other]
x_test = x_test[categorical + numeric + other]

Теперь используем `Pipeline`:
- с помощью него можно закодировать категориальные фичи с помощью `OneHotEncoder`
- нормализовать численные фичи с помощью `StandardScaler` или `MinMaxScaler`
- также если вы не заполнили наны, полезен бывает `SimpleImputer`

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

numeric_pipeline = Pipeline([
    # при желании можете добавить SimpleImputer
    # YOUR CODE HERE
    raise NotImplementedError()
    ('scaling', StandardScaler())
])

column_transformer = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown='ignore'), categorical),
    ('numeric', numeric_pipeline, numeric),
    ('other', 'passthrough', other)
])

# .toarray() необходим, если вы используете категориальные фичи
# в ином случае можете это удалить
x_train = column_transformer.fit_transform(x_train).toarray()
x_val = column_transformer.transform(x_val).toarray()
x_test = column_transformer.transform(x_test).toarray()

## Задание 5. Сравнение методов градиентного спуска (2 балла)

В этом задании вам предстоит сравнить методы градиентного спуска на подготовленных вами данных из предыдущего задания.

### Задание 5.1. Подбор оптимальной длины шага (1 балл)


Реализуйте функцию, котоаря принимает на вход `x_train, y_train, x_valid, y_valid, optimizer`, обучает `LinearRegression` и возвращает лосс на валидационном сете.

In [ ]:
from sklearn.metrics import r2_score
from tqdm import tqdm

In [ ]:
def train_model(x_train, y_train, x_valid, y_valid, optimizer: BaseDescent | None):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
_n, _d = 200, 5
np.random.seed(42)
_X_tm = np.random.randn(_n, _d)
_y_tm = _X_tm @ np.random.randn(_d) + np.random.randn(_n) * 0.1
_X_tm_val = np.random.randn(50, _d)
_y_tm_val = _X_tm_val @ np.random.randn(_d)

_opt = VanillaGradientDescent(lr_schedule=TimeDecayLR(lambda_=0.01))
_loss = train_model(_X_tm, _y_tm, _X_tm_val, _y_tm_val, _opt)
assert isinstance(_loss, (float, np.floating)), "train_model должна возвращать float"
assert not np.isnan(_loss), "Loss не должен быть NaN"
assert _loss > 0, "Loss должен быть положительным"

_loss_analytic = train_model(_X_tm, _y_tm, _X_tm_val, _y_tm_val, None)
assert isinstance(_loss_analytic, (float, np.floating)), "train_model должна возвращать float для optimizer=None"
assert not np.isnan(_loss_analytic), "Loss (аналитическое решение) не должен быть NaN"

_opt_sgd = StochasticGradientDescent(lr_schedule=TimeDecayLR(lambda_=0.01), batch_size=16)
_loss_sgd = train_model(_X_tm, _y_tm, _X_tm_val, _y_tm_val, _opt_sgd)
assert isinstance(_loss_sgd, (float, np.floating)), "train_model должна возвращать float для SGD"
assert not np.isnan(_loss_sgd), "Loss (SGD) не должен быть NaN"

Для каждого из четырёх методов (`VanillaGradientDescent`, `StochasticGradientDescent`, `MomentumDescent`, `Adam`) подберите наилучшую начальную длину шага $\lambda$ (`lambda_` в `TimeDecayLR`).

**Алгоритм:**
1. Задайте массив кандидатов `lambdas_` с помощью **логарифмической сетки** (`np.logspace`)
2. Для каждого метода переберите все значения `lambda_`:
   - создайте `TimeDecayLR(lambda_=lambda_)` и передайте в `optimizer`
   - вызовите `train_model(x_train, y_train, x_val, y_val, optimizer)` — она вернёт loss на валидации;
   - запомните пару `(lambda_, loss)`.
3. Выберите `best_lambda` — ту лямбду, при которой loss на валидации минимален
4. С лучшей лямбдой **заново обучите** модель на `x_train` и заполните словарь `report`:

```python
report[optimizer_name] = {
    "best_lambda": best_lambda,         # лучшая лямбда
    "r2_test": ...,                     # R² на тестовой выборке (x_test, y_test)
    "r2_train": ...,                    # R² на обучающей выборке
    "loss_history": model.loss_history, # история loss на обучении
    "loss_test": ...,                   # loss на тестовой выборке
    "loss_train": ...,                  # loss на обучающей выборке
}
```

Все гиперпараметры кроме `lambda_` оставьте по умолчанию (batch_size, alpha, и т.д.).

In [ ]:
optimizers = {
    "VanillaGradientDescent": VanillaGradientDescent, 
    "StochasticGradientDescent": StochasticGradientDescent, 
    "MomentumDescent": MomentumDescent, 
    "Adam": Adam
}
report = {
    "VanillaGradientDescent": {}, 
    "StochasticGradientDescent": {}, 
    "MomentumDescent": {}, 
    "Adam": {}
}

report = {optimizer_name: {
    "loss_history": None, 
    "loss_test": None, 
    "loss_train": None,
    "best_lambda": None, 
    "r2_test": None,
    "r2_train": None
    } for optimizer_name in optimizers}

In [ ]:
# lambdas_ = ...

# YOUR CODE HERE
raise NotImplementedError()

for optimizer_name, optimizer in optimizers.items():
    lambda2loss = dict()
    
    for lambda_ in tqdm(lambdas_):
    # YOUR CODE HERE
    raise NotImplementedError()


    best_lambda = min(lambda2loss, key=lambda2loss.get)
    # YOUR CODE HERE
    raise NotImplementedError()

За перебор 0,2 балла. За метрику R2 на тесте >= 0.85 - 0,4 балла, за метрику R2 на тесте >= 0.98 - 0,4 балла. Всего 1 балл.

In [ ]:
_required_keys = {"best_lambda", "r2_test", "r2_train", "loss_history", "loss_test", "loss_train"}

for opt_name in optimizers:
    assert opt_name in report
    entry = report[opt_name]
    for key in _required_keys:
        assert key in entry

    assert entry["best_lambda"] is not None
    assert entry["best_lambda"] > 0

    assert entry["r2_test"] is not None
    assert entry["r2_test"] > 0

    assert entry["loss_history"] is not None and len(entry["loss_history"]) > 1
    assert entry["loss_history"][-1] < entry["loss_history"][0]

    assert entry["loss_test"] is not None
    assert not np.isnan(entry["loss_test"])

In [ ]:
assert any(report[opt]["r2_test"] >= 0.85 for opt in optimizers), "No optimizer has r2 >= 0.85"

In [ ]:
assert any(report[opt]["r2_test"] >= 0.98 for opt in optimizers), "No optimizer has r2 >= 0.98"

### Задание 5.2. Сравнение методов (1 балл)

* Постройте график зависимости ошибки на обучающей выборке от номера итерации (все методы на одном графике).
* Постройте сравнительную таблицу каждого метода с лучшими параметрами и метриками (loss, r2)
* Балл будет выставлен именно за выводы. Если вы показали графики, но не сделали выводов - 0 баллов.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
optimizers = ["VanillaGradientDescent", "StochasticGradientDescent", "MomentumDescent", "Adam"]

# YOUR CODE HERE
raise NotImplementedError()


## Выводы
Напишите **в следующей ячейке** свои выводы в этой ячейке (где и вопрос) о сравнении: 
- какой метод набрал наивысший R2?
- какой метод сошелся быстрее остальных?
- отличаются ли порядком лямбды для каждого из методов?
- можете написать другие свои инсайты при анализе данных, что помогло повысить метрику? (дополнительно)

YOUR ANSWER HERE

## Задание 6. Стохастический градиентный спуск и размер батча (2 балла)

В этом задании вам предстоит исследовать влияние размера батча на работу стохастического градиентного спуска.

### Задание 6.1. Функция для замера эксперимента (0.5 балла)

Реализуйте функцию `run_sgd_experiment`, которая:
1. Создаёт `SGD` с заданным `batch_size` и `lr_schedule` и модель линейной регрессии.
2. Замеряет время обучения (используйте модуль `time`).
3. Считает количество итераций до сходимости — это `len(model.loss_history)`
4. Возвращает словарь `{"time": <float>, "iterations": <int>}`.


In [ ]:
import time


def run_sgd_experiment(
    X: np.ndarray,
    y: np.ndarray,
    batch_size: int,
    max_iter: int = 5000,
) -> dict:
    """
    Запускает SGD с заданным batch_size и возвращает словарь
    {"time": <время в секундах>, "iterations": <число итераций до сходимости>}.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
_test_n, _test_d = 200, 5
np.random.seed(42)
_X_test6 = np.random.randn(_test_n, _test_d)
_y_test6 = _X_test6 @ np.random.randn(_test_d) + np.random.randn(_test_n) * 0.1

result = run_sgd_experiment(_X_test6, _y_test6, batch_size=32)

assert isinstance(result, dict), "run_sgd_experiment должна возвращать dict"
assert "time" in result and "iterations" in result, \
    'Словарь должен содержать ключи "time" и "iterations"'
assert isinstance(result["time"], float), '"time" должен быть float (секунды)'
assert isinstance(result["iterations"], (int, np.integer)), '"iterations" должен быть int'
assert result["time"] > 0, "Время должно быть положительным"
assert result["iterations"] > 0, "Число итераций должно быть положительным"

r1 = run_sgd_experiment(_X_test6, _y_test6, batch_size=1, max_iter=500)
r_full = run_sgd_experiment(_X_test6, _y_test6, batch_size=_test_n, max_iter=500)
assert r1["iterations"] > 0 and r_full["iterations"] > 0

### Задание 6.2. Сбор статистики по размерам батча (0.5 балла)

Для каждого размера батча из списка `batch_sizes` сделайте $k$ запусков (`n_runs`) функции `run_sgd_experiment` и посчитайте **среднее** время и **среднее** число итераций.

Сохраните результаты в словарь `batch_stats`, где ключ  размер батча, а значение словарь `{"mean_time": ..., "mean_iterations": ...}`.


In [ ]:
n_runs = 10
batch_sizes = [1, 2, 4, 64, 128, 512, 1024, 2048, 4096, 10000, 20000]

# batch_stats: dict[int, dict] — {batch_size: {"mean_time": float, "mean_iterations": float}}
batch_stats = {}

# YOUR CODE HERE
raise NotImplementedError()


In [ ]:
assert isinstance(batch_stats, dict)
assert len(batch_stats) == len(batch_sizes)

for bs in batch_sizes:
    assert bs in batch_stats, f"Нет записи для batch_size={bs}"
    entry = batch_stats[bs]
    assert "mean_time" in entry, f'Нет ключа "mean_time" для batch_size={bs}'
    assert "mean_iterations" in entry, f'Нет ключа "mean_iterations" для batch_size={bs}'
    assert entry["mean_time"] > 0, f"mean_time должен быть > 0 для batch_size={bs}"
    assert entry["mean_iterations"] > 0, f"mean_iterations должен быть > 0 для batch_size={bs}"

### Задание 6.3. График: число итераций до сходимости vs размер батча (0 баллов, необходимо для выводов)

Постройте график зависимости среднего числа итераций до сходимости от размера батча.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Задание 6.4. График: время до сходимости vs размер батча (0 баллов, необходимо для выводов)

Постройте график зависимости среднего времени до сходимости от размера батча.


In [ ]:
# YOUR CODE HERE
raise NotImplementedError()


## Выводы о SGD (1 балл)
Напишите **в следующей ячейке** свои выводы о влиянии размера батча на работу SGD:
- Как размер батча влияет на **число итераций** до сходимости?
- Как размер батча влияет на **время** до сходимости? 
- Какой размер батча вы бы рекомендовали и почему? Существует ли «оптимальный» компромисс

YOUR ANSWER HERE

## Задание 7. Регуляризация (1 балл)

В этом задании вам предстоит исследовать влияние регуляризации на работу различных методов градиентного спуска. Напомним, регуляризация – это добавка к функции потерь, которая штрафует за норму весов. Мы будем использовать $L_2$-регуляризацию, таким образом функция потерь приобретает следующий вид:

$$
    Q(w) = \dfrac{1}{\ell} \sum\limits_{i=1}^{\ell} (a_w(x_i) - y_i)^2 + \dfrac{\mu}{2} \| w \|^2
$$

Вам необходимо вернуться выше в класc `LinearRegression` и добавить  `compute_loss` и `compute_gradients` необходимую часть с коэффицинтом.

### Задание 7.1. Функция обучения с регуляризацией (0 баллов, вспомогательная)

Реализуйте функцию `train_model_reg`, которая обучает `LinearRegression` с заданным оптимизатором и коэффициентом регуляризации `l2_coef` и возвращает loss на валидационной выборке.

Это расширение функции `train_model` из задания 5 с добавлением параметра `l2_coef`.


In [ ]:
def train_model_reg(
    x_train, y_train, x_valid, y_valid,
    optimizer: BaseDescent,
    l2_coef: float = 0.0,
) -> float:
    """
    Обучает LinearRegression с заданным optimizer и l2_coef.
    Возвращает loss на валидационной выборке.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
_n, _d = 200, 5
np.random.seed(42)
_X7 = np.random.randn(_n, _d)
_y7 = _X7 @ np.random.randn(_d) + np.random.randn(_n) * 0.1
_X7_val = np.random.randn(50, _d)
_y7_val = _X7_val @ np.random.randn(_d)

_opt = VanillaGradientDescent(lr_schedule=TimeDecayLR(lambda_=0.01))
_loss_no_reg = train_model_reg(_X7, _y7, _X7_val, _y7_val, _opt, l2_coef=0.0)
assert isinstance(_loss_no_reg, (float, np.floating)), "Функция должна возвращать float"
assert not np.isnan(_loss_no_reg), "Loss не должен быть NaN"

_opt2 = VanillaGradientDescent(lr_schedule=TimeDecayLR(lambda_=0.01))
_loss_with_reg = train_model_reg(_X7, _y7, _X7_val, _y7_val, _opt2, l2_coef=1.0)
assert isinstance(_loss_with_reg, (float, np.floating)), "Функция должна возвращать float"
assert not np.isnan(_loss_with_reg), "Loss не должен быть NaN"

assert _loss_no_reg != _loss_with_reg, \
    "Loss с l2_coef=0.0 и l2_coef=1.0 должны отличаться"

### Задание 7.2. Подбор лучших $\lambda$ и $\mu$ с регуляризацией (0.5 балла)

Для каждого из четырёх методов подберите по сетке лучшую пару гиперпараметров:
- `lambda_` — начальный шаг в `TimeDecayLR`
- `mu` — коэффициент $L_2$-регуляризации (`l2_coef` в `LinearRegression`)

Сохраните результаты в словарь `report_reg` той же структуры, что и `report` из задания 5, но с дополнительным ключом `"best_mu"`:

```python
report_reg[optimizer_name] = {
    "best_lambda": ...,
    "best_mu": ...,
    "r2_test": ...,
    "r2_train": ...,
    "loss_history": ...,        # loss_history на train
    "loss_test": ...,           # loss на тестовой выборке
    "loss_train": ...,
}
```

Для финального обучения с лучшими параметрами используйте полную обучающую выборку `x_train`.


In [ ]:
optimizers_reg = {
    "VanillaGradientDescent": VanillaGradientDescent,
    "StochasticGradientDescent": StochasticGradientDescent,
    "MomentumDescent": MomentumDescent,
    "Adam": Adam,
}

report_reg = {name: {"best_lambda": None, "best_mu": None, "r2_test": None, "r2_train": None, "loss_history": None, "loss_test": None, "loss_train": None}
              for name in optimizers_reg}

# YOUR CODE HERE
raise NotImplementedError()


In [ ]:
assert isinstance(report_reg, dict), "report_reg должен быть словарём"

_required_keys = {"best_lambda", "best_mu", "r2_test", "r2_train", "loss_history", "loss_test", "loss_train"}
for opt_name in optimizers_reg:
    assert opt_name in report_reg, f"Нет записи для {opt_name}"
    entry = report_reg[opt_name]
    for key in _required_keys:
        assert key in entry, f'Нет ключа "{key}" для {opt_name}'
    assert entry["r2_test"] is not None, f"r2_test не должен быть None для {opt_name}"
    assert entry["r2_test"] > 0, f"r2_test должен быть > 0 для {opt_name}"
    assert entry["best_lambda"] is not None, f"best_lambda не должен быть None для {opt_name}"
    assert entry["best_mu"] is not None, f"best_mu не должен быть None для {opt_name}"
    assert entry["best_mu"] > 0, f"best_mu должен быть > 0 для {opt_name}"
    assert entry["loss_history"] is not None and len(entry["loss_history"]) > 0, \
        f"loss_history не должен быть пустым для {opt_name}"


### Задание 7.3. Сравнение: регуляризация vs без регуляризации (0.5 балла за выводы)

Постройте для каждого метода **один график** с двумя кривыми loss на обучающей выборке:
- Без регуляризации (из `report` задания 5)
- С регуляризацией (из `report_reg`)

Всего должно получиться **4 графика** (по одному на метод). Не забудьте легенды и заголовки.


In [ ]:
# YOUR CODE HERE
raise NotImplementedError()


### Сравнительная таблица: R² и loss на тесте

Выведите таблицу, сравнивающую результаты каждого метода с регуляризацией и без. Включите в таблицу R2 на трейне и тесте, лосс на трейне и тесте.


In [ ]:
# YOUR CODE HERE
raise NotImplementedError()


## Выводы о регуляризации

Напишите **в следующей ячейке** свои выводы о влиянии регуляризации:
- Как регуляризация повлияла на **сходимость** (скорость, стабильность)?
- Как изменилось качество ($R^2$) на **обучающей** выборке?
- Как изменилось качество ($R^2$) на **тестовой** выборке? Чем вы можете это объяснить?

YOUR ANSWER HERE

### Фидбек

Можете оставить тут мем-ассоциацию с этим дз